#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:
import pickle
import xmlrpc.client
import xmlrpc
import pandas as pd
import os
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler

from datetime import datetime

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8018), 
                            requestHandler=RequestHandler)

with open("stats_server_setting.txt", "r") as setting_file:
    MAX_LINES = int(setting_file.readline())

def get_file_len(file):
    with open(file, "r") as f:
        return sum(1 for line in f)

def create_log_file(file_name):
    f = open(file_name, "w")
    f.write("type,datetime\n")
    return f


# Добавление строки в лог
def add_log(sname):
    current_time = datetime.datetime.now()
    log_format = current_time.strftime("%Y-%m-%d %H:%M:%S")
    file_format = current_time.strftime("%Y%m%d_%H%M%S")
    files = os.listdir("log")
    if files:
        # Получаем список файлов
        files = [os.path.join("log", f) for f in files]
        # Выбираем последний файл
        last_file = max(files)
        # Определяем количество строк в файле
        lines = get_file_len(last_file)
        if lines > MAX_LINES: 
              f = create_log_file("log/"+file_format+".csv")
        else: f = open(last_file, "a+")
    else: f = create_log_file("log/"+file_format+".csv")
    f.write(str(sname)+','+ log_format +'\n')
    f.close()
    return True

server.register_function(add_log, 'add_log')

datetime_format = "%Y-%m-%d %H:%M:%S"
filename_format = "log\%Y%m%d_%H%M%S.csv"

def get_files_by_date(start_date, end_date):
    files = os.listdir("log")
    if files:
        files = [os.path.join("log", f) for f in files]
        # Выбираем только те файлы, которые находятся в нужном временном диапазоне
        filtered = [file for file in files
                    if start_date.strftime(filename_format) <= file <= end_date.strftime(filename_format)]
        early_files = [file for file in files if file < start_date.strftime(filename_format)]
        if len(early_files) > 0:
            # Добавляем последний файл перед указанной датой
            last_file = early_files[-1]
            filtered.insert(0, last_file)
        return filtered
    
def get_log(start_date, end_date):
    files = get_files_by_date(start_date, end_date)
    logs = []
    for file in files:
        logs.append(pd.read_csv(file, sep=',', encoding='utf8'))
    log = pd.concat(logs, ignore_index=True)
    return log[(start_date.strftime(datetime_format) <= log['datetime']) & 
               (log['datetime'] <= end_date.strftime(datetime_format))]

def load_log(start_time_str, end_time_str, type):
    start_time = datetime.strptime(start_time_str, datetime_format)
    end_time = datetime.strptime(end_time_str, datetime_format)
    filter_log = get_log(start_time, end_time)
    # print(filter_log)
    filter_log = filter_log[filter_log['type'] == type]
    p = pickle.dumps(filter_log)        
    return xmlrpc.client.Binary(p)
    
server.register_function(load_log, 'load_log')

print ("Listening on port 8018...")
server.serve_forever()


Listening on port 8018...
